## 0. Install and Import Dependencies

In [1]:
!pip install -q mediapipe opencv-python pandas scikit-learn


[notice] A new release of pip is available: 23.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


## 1. Make Some Detections

In [1]:
import mediapipe as mp
import cv2 as cv

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [18]:
cap = cv.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = holistic.process(image)
        # print(results.pose_landmarks)

        # Recolor Feed Back to BGR
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

        # Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                  mp_drawing.DrawingSpec(color=(80,110,0), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                  )


        # Draw Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))


        # # Draw Left hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))

        # # Draw pose landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        

        cv.imshow("Raw Webcam Feed", image)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv.destroyAllWindows()

In [16]:
results.pose_landmarks.landmark[0].visibility

0.9999204277992249

## 2. Capture Landmarks & Export to CSV
<!-- <img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" > -->

In [17]:
import csv
import numpy as np
import os

In [24]:
num_coords = len(results.face_landmarks.landmark) + len(results.pose_landmarks.landmark)

In [26]:
landmarks = ["class"]

for val in range(1, num_coords+1):
    landmarks += [f"x{val}", f"y{val}", f"z{val}", f"v{val}"]

In [28]:
# landmarks

In [48]:
with open("dataset/coords.csv", mode="w", newline="") as f:
    csv_writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [131]:
class_name = "Namaste"

In [132]:
cap = cv.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = holistic.process(image)
        # print(results.pose_landmarks)

        # Recolor Feed Back to BGR
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

        # Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                  mp_drawing.DrawingSpec(color=(80,110,0), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                  )


        # Draw Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))


        # # Draw Left hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))

        # # Draw pose landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        
        # Export coordinates
        try:
            # Extracting pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())


            # Extracting face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())


            row = pose_row + face_row
            row.insert(0, class_name)

            with open("dataset/coords.csv", mode="a", newline="") as f:
                csv_writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
        except:
            pass

        cv.imshow("Raw Webcam Feed", image)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv.destroyAllWindows()

## 3. Train Custom Model Using Scikit Learn

### 3.1 Read in collected Data and Process

In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [133]:
df = pd.read_csv("dataset/coords.csv")

In [134]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Happy,0.635784,0.335637,-0.573722,0.999984,0.658159,0.263864,-0.541347,0.999976,0.669439,...,-0.011240,0.0,0.675851,0.277081,-0.002050,0.0,0.679620,0.272606,-0.002188,0.0
1,Happy,0.635479,0.337061,-0.587487,0.999985,0.657810,0.266311,-0.559996,0.999976,0.669439,...,-0.011758,0.0,0.678713,0.280609,-0.002558,0.0,0.682368,0.276862,-0.002744,0.0
2,Happy,0.635490,0.337222,-0.534357,0.999986,0.657791,0.267417,-0.506601,0.999978,0.669555,...,-0.012499,0.0,0.681021,0.283317,-0.003306,0.0,0.684664,0.279227,-0.003492,0.0
3,Happy,0.635487,0.338835,-0.529565,0.999987,0.657739,0.269145,-0.504197,0.999979,0.669608,...,-0.013219,0.0,0.681078,0.287396,-0.003861,0.0,0.684776,0.283402,-0.004087,0.0
4,Happy,0.635532,0.339729,-0.529663,0.999987,0.657735,0.269685,-0.503234,0.999980,0.669744,...,-0.012790,0.0,0.681979,0.288398,-0.003042,0.0,0.685609,0.284299,-0.003225,0.0


In [135]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
810,Namaste,0.543094,0.299570,-0.546627,0.999988,0.564499,0.239743,-0.506168,0.999973,0.575270,...,-0.005321,0.0,0.577368,0.239827,0.007933,0.0,0.580936,0.235185,0.008140,0.0
811,Namaste,0.539995,0.296582,-0.548678,0.999988,0.561982,0.237534,-0.509313,0.999974,0.573723,...,-0.005348,0.0,0.575988,0.238207,0.008819,0.0,0.579586,0.233640,0.009074,0.0
812,Namaste,0.537238,0.295214,-0.549406,0.999989,0.558443,0.236485,-0.509924,0.999975,0.571307,...,-0.005598,0.0,0.573612,0.238402,0.007860,0.0,0.577267,0.233750,0.008070,0.0
813,Namaste,0.532352,0.295259,-0.552352,0.999989,0.553996,0.236352,-0.513155,0.999977,0.568129,...,-0.005868,0.0,0.571261,0.236189,0.007650,0.0,0.574969,0.231121,0.007865,0.0
814,Namaste,0.527562,0.295389,-0.564558,0.999990,0.548878,0.236436,-0.525976,0.999978,0.564119,...,-0.006273,0.0,0.569095,0.234466,0.007382,0.0,0.572797,0.229682,0.007555,0.0


In [67]:
df[df["class"] == "Sad"]

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
228,Sad,0.551332,0.574337,-0.854185,0.999692,0.574413,0.504267,-0.852872,0.999291,0.587177,...,-0.025622,0.0,0.602083,0.520961,-0.021060,0.0,0.605842,0.513629,-0.021968,0.0
229,Sad,0.550090,0.577942,-0.582554,0.999713,0.573099,0.507530,-0.583860,0.999342,0.586736,...,-0.026340,0.0,0.602319,0.519215,-0.022637,0.0,0.605677,0.512302,-0.023492,0.0
230,Sad,0.549722,0.579780,-0.564631,0.999732,0.572434,0.509329,-0.566943,0.999388,0.586485,...,-0.025783,0.0,0.601450,0.516284,-0.022229,0.0,0.604748,0.509899,-0.023169,0.0
231,Sad,0.549095,0.581399,-0.561325,0.999748,0.571318,0.511218,-0.564049,0.999428,0.585670,...,-0.025559,0.0,0.601351,0.515385,-0.022296,0.0,0.604482,0.509257,-0.023261,0.0
232,Sad,0.547642,0.581980,-0.542092,0.999762,0.569587,0.512030,-0.545448,0.999462,0.584106,...,-0.025224,0.0,0.600053,0.514153,-0.022123,0.0,0.603199,0.508271,-0.023101,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,Sad,0.545442,0.565303,-0.557456,0.999886,0.569036,0.491705,-0.554899,0.999797,0.583990,...,-0.023656,0.0,0.589935,0.495015,-0.022392,0.0,0.593206,0.487327,-0.023347,0.0
389,Sad,0.545384,0.565896,-0.557639,0.999889,0.568949,0.492684,-0.555307,0.999804,0.583902,...,-0.023813,0.0,0.590883,0.494539,-0.022720,0.0,0.594193,0.487118,-0.023725,0.0
390,Sad,0.545811,0.566434,-0.577509,0.999890,0.569127,0.492831,-0.572117,0.999806,0.584149,...,-0.023502,0.0,0.592953,0.493943,-0.021982,0.0,0.596245,0.486397,-0.022940,0.0
391,Sad,0.545963,0.566979,-0.583011,0.999892,0.569256,0.493053,-0.577663,0.999810,0.584297,...,-0.022755,0.0,0.592638,0.492138,-0.021130,0.0,0.595846,0.484906,-0.022093,0.0


In [136]:
x = df.drop("class", axis=1)
y = df["class"]

In [69]:
x

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,0.635784,0.335637,-0.573722,0.999984,0.658159,0.263864,-0.541347,0.999976,0.669439,0.262146,...,-0.011240,0.0,0.675851,0.277081,-0.002050,0.0,0.679620,0.272606,-0.002188,0.0
1,0.635479,0.337061,-0.587487,0.999985,0.657810,0.266311,-0.559996,0.999976,0.669439,0.264625,...,-0.011758,0.0,0.678713,0.280609,-0.002558,0.0,0.682368,0.276862,-0.002744,0.0
2,0.635490,0.337222,-0.534357,0.999986,0.657791,0.267417,-0.506601,0.999978,0.669555,0.265698,...,-0.012499,0.0,0.681021,0.283317,-0.003306,0.0,0.684664,0.279227,-0.003492,0.0
3,0.635487,0.338835,-0.529565,0.999987,0.657739,0.269145,-0.504197,0.999979,0.669608,0.267345,...,-0.013219,0.0,0.681078,0.287396,-0.003861,0.0,0.684776,0.283402,-0.004087,0.0
4,0.635532,0.339729,-0.529663,0.999987,0.657735,0.269685,-0.503234,0.999980,0.669744,0.267729,...,-0.012790,0.0,0.681979,0.288398,-0.003042,0.0,0.685609,0.284299,-0.003225,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,0.593291,0.460913,-0.439111,0.999991,0.612543,0.403090,-0.412169,0.999985,0.623231,0.401068,...,-0.006668,0.0,0.630162,0.397924,0.001983,0.0,0.633172,0.393702,0.002029,0.0
529,0.599210,0.457863,-0.465936,0.999992,0.617803,0.402253,-0.437028,0.999985,0.626324,0.400670,...,-0.005354,0.0,0.630800,0.394416,0.003695,0.0,0.633895,0.389935,0.003844,0.0
530,0.600542,0.449213,-0.442148,0.999992,0.618557,0.394756,-0.411470,0.999986,0.626827,0.394223,...,-0.004477,0.0,0.628250,0.386615,0.004900,0.0,0.631409,0.381982,0.005079,0.0
531,0.600241,0.447358,-0.433186,0.999992,0.618251,0.392130,-0.403264,0.999986,0.626746,0.391575,...,-0.005035,0.0,0.624990,0.383098,0.004675,0.0,0.628049,0.378559,0.004824,0.0


In [70]:
y

0           Happy
1           Happy
2           Happy
3           Happy
4           Happy
          ...    
528    Victorious
529    Victorious
530    Victorious
531    Victorious
532    Victorious
Name: class, Length: 533, dtype: object

In [137]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234)

In [138]:
y_train

398         Victorious
475         Victorious
623    Wakanda Forever
93               Happy
221              Happy
            ...       
372                Sad
204              Happy
53               Happy
294                Sad
723            Namaste
Name: class, Length: 570, dtype: object

### 3.2 Train Machine Learning Classification Model

In [76]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [141]:
pipelines = {
    "lr": make_pipeline(StandardScaler(), LogisticRegression(max_iter=1200000)),
    "rc": make_pipeline(StandardScaler(), RidgeClassifier()),
    "rf": make_pipeline(StandardScaler(), RandomForestClassifier()),
    "gb": make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

In [142]:
fit_models = {}

for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [143]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression(max_iter=1200000))]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [144]:
fit_models["lr"].predict(X_test)

array(['Wakanda Forever', 'Namaste', 'Sad', 'Victorious', 'Sad',
       'Namaste', 'Namaste', 'Namaste', 'Victorious', 'Victorious',
       'Happy', 'Happy', 'Victorious', 'Happy', 'Happy',
       'Wakanda Forever', 'Sad', 'Namaste', 'Happy', 'Happy', 'Happy',
       'Happy', 'Victorious', 'Namaste', 'Happy', 'Happy', 'Happy',
       'Victorious', 'Happy', 'Sad', 'Namaste', 'Happy', 'Sad', 'Happy',
       'Namaste', 'Happy', 'Happy', 'Wakanda Forever', 'Wakanda Forever',
       'Happy', 'Happy', 'Namaste', 'Victorious', 'Happy',
       'Wakanda Forever', 'Sad', 'Namaste', 'Namaste', 'Wakanda Forever',
       'Victorious', 'Sad', 'Sad', 'Happy', 'Wakanda Forever',
       'Wakanda Forever', 'Wakanda Forever', 'Victorious', 'Victorious',
       'Wakanda Forever', 'Happy', 'Wakanda Forever', 'Victorious',
       'Happy', 'Sad', 'Happy', 'Namaste', 'Victorious',
       'Wakanda Forever', 'Sad', 'Victorious', 'Victorious',
       'Wakanda Forever', 'Wakanda Forever', 'Happy', 'Sad', 'Happy',

### 3.3 Evaluate and Serialize Model

In [96]:
from sklearn.metrics import accuracy_score
import pickle

In [145]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [146]:
with open("models/body_language.pkl", "wb") as f:
    pickle.dump(fit_models["lr"], f)

## 4. Make Detections with Model

In [147]:
with open("models/body_language.pkl", "rb") as f:
    model = pickle.load(f)

In [115]:
tuple(np.multiply(
    np.array(
        (
            results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
            results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y
        )
    ),
    [640, 480]).astype(int)
)

(464, 194)

In [148]:
cap = cv.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = holistic.process(image)

        # Predict class
        try:
            # Extracting pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())


            # Extracting face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())


            row = pose_row + face_row

            X = pd.DataFrame([row], columns=df.columns.values[1:])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            # print(body_language_class, body_language_prob)

            # Grab Ear coords
            ear_coords = tuple(np.multiply(
                np.array(
                    (
                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y
                    )
                ),
                [1280, 720]).astype(int)
            )
            
            cv.rectangle(
                image, 
                (ear_coords[0], ear_coords[1]+5), 
                (ear_coords[0]+len(body_language_class)*20, ear_coords[1]-30), 
                (255, 255, 255), -1)
            
            cv.putText(image, body_language_class, ear_coords, cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv.LINE_AA)

            # Get status box
            cv.rectangle(image, (0, 0), (250, 60), (245, 117, 16), -1)

            # Display Class
            cv.putText(image, "CLASS", (95, 12), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv.LINE_AA)
            cv.putText(image, body_language_class.split(" ")[0], (90, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)

            # Display Probability
            cv.putText(image, "PROB", (15, 12), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv.LINE_AA)
            cv.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)], 2)), (10, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)


        except:
            pass


        # Recolor Feed Back to BGR
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

        # Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                  mp_drawing.DrawingSpec(color=(80,110,0), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                  )


        # Draw Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))


        # # Draw Left hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))

        # # Draw pose landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        
        cv.imshow("Raw Webcam Feed", image)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv.destroyAllWindows()

0.54